## You're here! 
Welcome to your first competition in the [ITI's AI Pro training program](https://ai.iti.gov.eg/epita/ai-engineer/)! We hope you enjoy and learn as much as we did prepairing this competition.


## Introduction

In the competition, it's required to predict the `Severity` of a car crash given info about the crash, e.g., location.

This is the getting started notebook. Things are kept simple so that it's easier to understand the steps and modify it.

Feel free to `Fork` this notebook and share it with your modifications **OR** use it to create your submissions.

### Prerequisites
You should know how to use python and a little bit of Machine Learning. You can apply the techniques you learned in the training program and submit the new solutions! 

### Checklist
You can participate in this competition the way you perefer. However, I recommend following these steps if this is your first time joining a competition on Kaggle.

* Fork this notebook and run the cells in order.
* Submit this solution.
* Make changes to the data processing step as you see fit.
* Submit the new solutions.

*You can submit up to 5 submissions per day. You can select only one of the submission you make to be considered in the final ranking.*


Don't hesitate to leave a comment or contact me if you have any question!

## Import the libraries

We'll use `pandas` to load and manipulate the data. Other libraries will be imported in the relevant sections.

In [1]:
%matplotlib inline
from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
import os
import xml.etree.ElementTree as et

In [2]:
labelencoder = LabelEncoder()

## Exploratory Data Analysis
In this step, one should load the data and analyze it. However, I'll load the data and do minimal analysis. You are encouraged to do thorough analysis!

Let's load the data using `pandas` and have a look at the generated `DataFrame`.

In [3]:
dataset_path = '/kaggle/input/car-crashes-severity-prediction/'

In [4]:
df = pd.read_csv(os.path.join(dataset_path, 'train.csv'))
print("The shape of the dataset is {}.\n\n".format(df.shape))
df.head()

The shape of the dataset is (6407, 16).




,ID,Lat,Lng,Bump,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Stop,Amenity,Side,Severity,timestamp
0,0,37.762150,-122.405660,False,0.044,False,False,False,False,False,False,False,True,R,2,2016-03-25 15:13:02
1,1,37.719157,-122.448254,False,0.000,False,False,False,False,False,False,False,False,R,2,2020-05-05 19:23:00
2,2,37.808498,-122.366852,False,0.000,False,False,False,False,False,False,True,False,R,3,2016-09-16 19:57:16
3,3,37.785930,-122.391080,False,0.009,False,False,True,False,False,False,False,False,R,1,2020-03-29 19:48:43
4,4,37.719141,-122.448457,False,0.000,False,False,False,False,False,False,False,False,R,2,2019-10-09 08:47:00


We've got 6407 examples in the dataset with 14 featues, 1 ID, and the `Severity` of the crash.

By looking at the features and a sample from the data, the features look of numerical and catogerical types. What about some descriptive statistics?

In [5]:
df.drop(columns='ID').describe()

,Lat,Lng,Distance(mi),Severity
count,6407.000000,6407.000000,6407.000000,6407.000000
mean,37.765653,-122.405990,0.135189,2.293429
std,0.032555,0.028275,0.396360,0.521225
min,37.609619,-122.510440,0.000000,1.000000
25%,37.737096,-122.412210,0.000000,2.000000
50%,37.768238,-122.404835,0.000000,2.000000
75%,37.787813,-122.392477,0.041000,3.000000
max,37.825626,-122.349734,6.820000,4.000000


The output shows desciptive statistics for the numerical features, `Lat`, `Lng`, `Distance(mi)`, and `Severity`. I'll use the numerical features to demonstrate how to train the model and make submissions. **However you shouldn't use the numerical features only to make the final submission if you want to make it to the top of the leaderboard.**

### Remove columns with now effect

In [6]:
df.drop(columns=['Bump','Roundabout'],inplace=True)

#### Format date data in the dataframe and extract the hour to match with weather data

In [7]:
df['timestamp']=pd.to_datetime(df['timestamp'])
df['date_time']=pd.to_datetime(df['timestamp'].dt.strftime('%Y-%m-%d %H:00:00'))
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6407 entries, 0 to 6406
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   ID            6407 non-null   int64         
 1   Lat           6407 non-null   float64       
 2   Lng           6407 non-null   float64       
 3   Distance(mi)  6407 non-null   float64       
 4   Crossing      6407 non-null   bool          
 5   Give_Way      6407 non-null   bool          
 6   Junction      6407 non-null   bool          
 7   No_Exit       6407 non-null   bool          
 8   Railway       6407 non-null   bool          
 9   Stop          6407 non-null   bool          
 10  Amenity       6407 non-null   bool          
 11  Side          6407 non-null   object        
 12  Severity      6407 non-null   int64         
 13  timestamp     6407 non-null   datetime64[ns]
 14  date_time     6407 non-null   datetime64[ns]
dtypes: bool(7), datetime64[ns](2), float64

#### convert bool values to numeric int values for cleaning

In [8]:
df['Crossing'] = df['Crossing'].astype(int)
df['Give_Way'] = df['Give_Way'].astype(int)
df['Junction'] = df['Junction'].astype(int)
df['No_Exit'] = df['No_Exit'].astype(int)
df['Railway'] = df['Railway'].astype(int)
df['Stop'] = df['Stop'].astype(int)
df['Amenity'] = df['Amenity'].astype(int)

In [9]:
df = df[df.Severity.isin([2,3])]

#### encode side data as numeric

In [10]:
df['Side'] = labelencoder.fit_transform(df['Side'])

#### Clustring the lat and lng

In [11]:
# kmeans = KMeans(n_clusters = 3, init ='k-means++')
# kmeans.fit(df[df.columns[1:3]])
# df['lat_lang_clust'] = kmeans.fit_predict(df[df.columns[1:3]])
# centers = kmeans.cluster_centers_ 
# labels = kmeans.predict(df[df.columns[1:3]])
# df_merged.drop(columns=['Lat','Lng'],inplace=True)
# df.head(10)

In [12]:
df.Severity.value_counts()

2    4346
3    1855
Name: Severity, dtype: int64

In [13]:
df.head()

,ID,Lat,Lng,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Stop,Amenity,Side,Severity,timestamp,date_time
0,0,37.762150,-122.405660,0.044,0,0,0,0,0,0,1,1,2,2016-03-25 15:13:02,2016-03-25 15:00:00
1,1,37.719157,-122.448254,0.000,0,0,0,0,0,0,0,1,2,2020-05-05 19:23:00,2020-05-05 19:00:00
2,2,37.808498,-122.366852,0.000,0,0,0,0,0,1,0,1,3,2016-09-16 19:57:16,2016-09-16 19:00:00
4,4,37.719141,-122.448457,0.000,0,0,0,0,0,0,0,1,2,2019-10-09 08:47:00,2019-10-09 08:00:00
5,5,37.769611,-122.416878,0.000,0,0,0,0,0,0,0,1,2,2019-09-22 08:45:38,2019-09-22 08:00:00


### Import Holiday Data

In [14]:
xtree = et.parse(os.path.join(dataset_path, 'holidays.xml'))
xroot = xtree.getroot()
df_cols = ["date", "description"]
rows = []
for node in xroot:
    date = node.find("date").text if node is not None else None
    description = node.find("description").text if node is not None else None
    rows.append({"date": date,"description": description})
holidays_df = pd.DataFrame(rows)
holidays_df.head()

,date,description
0,2012-01-02,New Year Day
1,2012-01-16,Martin Luther King Jr. Day
2,2012-02-20,Presidents Day (Washingtons Birthday)
3,2012-05-28,Memorial Day
4,2012-07-04,Independence Day


#### format date to match with the train data

In [15]:
holidays_df['date']=pd.to_datetime(holidays_df['date'])

#### Check if the day of the accident was holiday or not

In [16]:
df['is_holiday']=df['timestamp'].dt.date.isin(holidays_df['date'].dt.date).astype(int)  
df['is_holiday']=df['timestamp'].dt.day_name().isin(['Saturday','Sunday']).astype(int)

### Import Weather Data

In [17]:
import numpy as np
weather_df = pd.read_csv(os.path.join(dataset_path, 'weather-sfcsv.csv'))

In [18]:
weather_df.describe()

,Year,Day,Month,Hour,Wind_Chill(F),Precipitation(in),Temperature(F),Humidity(%),Wind_Speed(mph),Visibility(mi)
count,6901.000000,6901.000000,6901.000000,6901.000000,3292.000000,3574.000000,6899.000000,6899.000000,6556.000000,6900.000000
mean,2018.293001,15.624837,6.775250,12.789886,59.762515,0.006444,59.905465,68.570952,10.813164,9.441851
std,1.390524,8.703753,3.567982,5.874155,9.338456,0.028822,7.977860,16.180126,6.406376,1.640513
min,2016.000000,1.000000,1.000000,0.000000,31.100000,0.000000,36.000000,10.000000,0.000000,0.000000
25%,2017.000000,8.000000,4.000000,8.000000,54.000000,0.000000,54.000000,59.000000,5.800000,10.000000
50%,2019.000000,15.000000,7.000000,14.000000,59.000000,0.000000,59.000000,70.000000,10.000000,10.000000
75%,2020.000000,23.000000,10.000000,17.000000,66.000000,0.000000,64.900000,80.000000,15.000000,10.000000
max,2020.000000,31.000000,12.000000,23.000000,98.000000,0.490000,98.100000,100.000000,40.300000,10.000000


In [19]:
weather_df['date_time']=pd.to_datetime(weather_df[["Year", "Month", "Day","Hour"]])
weather_df.dropna(subset=["Weather_Condition"],inplace=True)

In [20]:
weather_df['Weather_Condition'] = labelencoder.fit_transform(weather_df['Weather_Condition'])

In [21]:
weather_df['Visibility(mi)'].fillna(weather_df['Visibility(mi)'].mean(),inplace=True)
weather_df['Humidity(%)'].fillna(weather_df['Humidity(%)'].mean(),inplace=True)
weather_df['Temperature(F)'].fillna(weather_df['Temperature(F)'].mean(),inplace=True)
weather_df['Wind_Speed(mph)'].fillna(weather_df['Wind_Speed(mph)'].mean(),inplace=True)

In [22]:
weather_df.drop(columns=['Precipitation(in)','Selected','Wind_Chill(F)'],inplace=True)

In [23]:
weather_df.drop_duplicates(subset=['date_time'],inplace=True,keep='last')

In [24]:
weather_df.head()

,Year,Day,Month,Hour,Weather_Condition,Temperature(F),Humidity(%),Wind_Speed(mph),Visibility(mi),date_time
0,2020,27,7,18,3,64.0,70.0,20.0,10.0,2020-07-27 18:00:00
1,2017,30,9,17,17,71.1,57.0,9.2,10.0,2017-09-30 17:00:00
2,2017,27,6,5,16,57.9,87.0,15.0,9.0,2017-06-27 05:00:00
3,2016,7,9,9,0,66.9,73.0,4.6,10.0,2016-09-07 09:00:00
8,2019,14,2,15,14,55.0,64.0,18.4,10.0,2019-02-14 15:00:00


In [25]:
df_merged =pd.merge(df, weather_df, how='left', on=['date_time'])

In [26]:
df_merged.dropna(subset=['Year'],inplace=True)

In [27]:
df_merged.drop(columns=['timestamp','date_time'],inplace=True)

In [28]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6200 entries, 0 to 6200
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ID                 6200 non-null   int64  
 1   Lat                6200 non-null   float64
 2   Lng                6200 non-null   float64
 3   Distance(mi)       6200 non-null   float64
 4   Crossing           6200 non-null   int64  
 5   Give_Way           6200 non-null   int64  
 6   Junction           6200 non-null   int64  
 7   No_Exit            6200 non-null   int64  
 8   Railway            6200 non-null   int64  
 9   Stop               6200 non-null   int64  
 10  Amenity            6200 non-null   int64  
 11  Side               6200 non-null   int64  
 12  Severity           6200 non-null   int64  
 13  is_holiday         6200 non-null   int64  
 14  Year               6200 non-null   float64
 15  Day                6200 non-null   float64
 16  Month              6200 

In [29]:
df_merged.drop(columns=["ID"]).describe()

,Lat,Lng,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Stop,Amenity,...,is_holiday,Year,Day,Month,Hour,Weather_Condition,Temperature(F),Humidity(%),Wind_Speed(mph),Visibility(mi)
count,6200.000000,6200.000000,6200.000000,6200.000000,6200.000000,6200.000000,6200.000000,6200.000000,6200.000000,6200.000000,...,6200.000000,6200.000000,6200.000000,6200.000000,6200.000000,6200.000000,6200.000000,6200.000000,6200.000000,6200.000000
mean,37.765535,-122.406148,0.136023,0.082097,0.000484,0.237903,0.000161,0.026129,0.100806,0.036613,...,0.184516,2018.365484,15.691935,6.813710,12.826452,11.091129,59.911194,68.166221,10.813835,9.441689
std,0.032522,0.028279,0.390863,0.274534,0.021994,0.425834,0.012700,0.159532,0.301097,0.187825,...,0.387936,1.370054,8.745340,3.594906,5.813986,6.951141,8.016504,16.279713,6.370120,1.642199
min,37.609619,-122.507191,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,2016.000000,1.000000,1.000000,0.000000,0.000000,36.000000,10.000000,0.000000,0.120000
25%,37.736788,-122.412227,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,2017.000000,8.000000,4.000000,8.000000,3.000000,54.000000,59.000000,6.000000,10.000000
50%,37.768227,-122.404835,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,2019.000000,16.000000,7.000000,14.000000,14.000000,59.000000,70.000000,10.400000,10.000000
75%,37.787411,-122.392571,0.044000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,2020.000000,23.000000,10.000000,17.000000,17.000000,64.900000,80.000000,15.000000,10.000000
max,37.825626,-122.349734,5.890000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,2020.000000,31.000000,12.000000,23.000000,25.000000,98.000000,100.000000,40.300000,10.000000


## Data Splitting

Now it's time to split the dataset for the training step. Typically the dataset is split into 3 subsets, namely, the training, validation and test sets. In our case, the test set is already predefined. So we'll split the "training" set into training and validation sets with 0.8:0.2 ratio. 

*Note: a good way to generate reproducible results is to set the seed to the algorithms that depends on randomization. This is done with the argument `random_state` in the following command* 

In [30]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(df_merged, test_size=0.2, random_state=42) # Try adding `stratify` here

X_train = train_df.drop(columns=['ID', 'Severity'])
y_train = train_df['Severity']

X_val = val_df.drop(columns=['ID', 'Severity']) 
y_val = val_df['Severity']

As pointed out eariler, I'll use the numerical features to train the classifier. **However, you shouldn't use the numerical features only to make the final submission if you want to make it to the top of the leaderboard.** 

In [31]:
X_train = X_train[['Lat','Lng', 'Crossing', 'Stop','Weather_Condition','is_holiday']]

X_val = X_val[['Lat','Lng', 'Crossing', 'Stop','Weather_Condition','is_holiday']]

## Model Training

Let's train a model with the data! We'll train a Random Forest Classifier to demonstrate the process of making submissions. 

In [32]:
from sklearn.ensemble import RandomForestClassifier

# Create an instance of the classifier
classifier = RandomForestClassifier(max_depth=2,random_state=0)

# Train the classifier
classifier = classifier.fit(X_train, y_train)

Now let's test our classifier on the validation dataset and see the accuracy.

In [33]:
print("The accuracy of the classifier on the validation set is ", (classifier.score(X_val, y_val)))

The accuracy of the classifier on the validation set is  0.7612903225806451


Well. That's a good start, right? A classifier that predicts all examples' `Severity` as 2 will get around 0.63. You should get better score as you add more features and do better data preprocessing.

## Submission File Generation

We have built a model and we'd like to submit our predictions on the test set! In order to do that, we'll load the test set, predict the class and save the submission file. 

First, we'll load the data.

In [34]:
test_df = pd.read_csv(os.path.join(dataset_path, 'test.csv'))
test_df.head()

,ID,Lat,Lng,Bump,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Stop,Amenity,Side,timestamp
0,6407,37.786060,-122.390900,False,0.039,False,False,True,False,False,False,False,False,R,2016-04-04 19:20:31
1,6408,37.769609,-122.415057,False,0.202,False,False,False,False,False,False,False,False,R,2020-10-28 11:51:00
2,6409,37.807495,-122.476021,False,0.000,False,False,False,False,False,False,False,False,R,2019-09-09 07:36:45
3,6410,37.761818,-122.405869,False,0.000,False,False,True,False,False,False,False,False,R,2019-08-06 15:46:25
4,6411,37.732350,-122.414100,False,0.670,False,False,False,False,False,False,False,False,R,2018-10-17 09:54:58


Note that the test set has the same features and doesn't have the `Severity` column.
At this stage one must **NOT** forget to apply the same processing done on the training set on the features of the test set.

Now we'll add `Severity` column to the test `DataFrame` and add the values of the predicted class to it.

**I'll select the numerical features here as I did in the training set. DO NOT forget to change this step as you change the preprocessing of the training data.**

In [35]:
test_df.drop(columns=['Bump','Roundabout'],inplace=True)

test_df['timestamp']=pd.to_datetime(test_df['timestamp'])
test_df['date_time']=pd.to_datetime(test_df['timestamp'].dt.strftime('%Y-%m-%d %H:00:00'))


test_df['Crossing'] = test_df['Crossing'].astype(int)
test_df['Give_Way'] = test_df['Give_Way'].astype(int)
test_df['Junction'] = test_df['Junction'].astype(int)
test_df['No_Exit'] = test_df['No_Exit'].astype(int)
test_df['Railway'] = test_df['Railway'].astype(int)
test_df['Stop'] = test_df['Stop'].astype(int)
test_df['Amenity'] = test_df['Amenity'].astype(int)

test_df['Side'] = labelencoder.fit_transform(test_df['Side'])

test_df['is_holiday']=test_df['timestamp'].dt.date.isin(holidays_df['date'].dt.date).astype(int)  

test_df_merged =pd.merge(test_df, weather_df, how='left', on=['date_time'])

test_df_merged.dropna(inplace=True)

test_df_merged.drop(columns=['date_time','timestamp'],inplace=True)
test_df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1601 entries, 0 to 1600
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ID                 1601 non-null   int64  
 1   Lat                1601 non-null   float64
 2   Lng                1601 non-null   float64
 3   Distance(mi)       1601 non-null   float64
 4   Crossing           1601 non-null   int64  
 5   Give_Way           1601 non-null   int64  
 6   Junction           1601 non-null   int64  
 7   No_Exit            1601 non-null   int64  
 8   Railway            1601 non-null   int64  
 9   Stop               1601 non-null   int64  
 10  Amenity            1601 non-null   int64  
 11  Side               1601 non-null   int64  
 12  is_holiday         1601 non-null   int64  
 13  Year               1601 non-null   int64  
 14  Day                1601 non-null   int64  
 15  Month              1601 non-null   int64  
 16  Hour               1601 

In [36]:
test_df_merged.head()

,ID,Lat,Lng,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Stop,...,is_holiday,Year,Day,Month,Hour,Weather_Condition,Temperature(F),Humidity(%),Wind_Speed(mph),Visibility(mi)
0,6407,37.786060,-122.390900,0.039,0,0,1,0,0,0,...,0,2016,4,4,19,17,63.0,60.0,10.4,10.0
1,6408,37.769609,-122.415057,0.202,0,0,0,0,0,0,...,0,2020,28,10,11,3,65.0,56.0,5.0,9.0
2,6409,37.807495,-122.476021,0.000,0,0,0,0,0,0,...,0,2019,9,9,7,14,59.0,87.0,14.0,10.0
3,6410,37.761818,-122.405869,0.000,0,0,1,0,0,0,...,0,2019,6,8,15,3,72.0,59.0,17.0,10.0
4,6411,37.732350,-122.414100,0.670,0,0,0,0,0,0,...,0,2018,17,10,9,22,57.0,77.0,5.8,10.0


In [37]:
X_test = test_df_merged.drop(columns=['ID'])

# You should update/remove the next line once you change the features used for training
X_test = X_test[['Lat','Lng', 'Crossing', 'Stop','Weather_Condition','is_holiday']]
X_test.isna().sum()
y_test_predicted = classifier.predict(X_test)

test_df['Severity'] = y_test_predicted

test_df_merged.head()

,ID,Lat,Lng,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Stop,...,is_holiday,Year,Day,Month,Hour,Weather_Condition,Temperature(F),Humidity(%),Wind_Speed(mph),Visibility(mi)
0,6407,37.786060,-122.390900,0.039,0,0,1,0,0,0,...,0,2016,4,4,19,17,63.0,60.0,10.4,10.0
1,6408,37.769609,-122.415057,0.202,0,0,0,0,0,0,...,0,2020,28,10,11,3,65.0,56.0,5.0,9.0
2,6409,37.807495,-122.476021,0.000,0,0,0,0,0,0,...,0,2019,9,9,7,14,59.0,87.0,14.0,10.0
3,6410,37.761818,-122.405869,0.000,0,0,1,0,0,0,...,0,2019,6,8,15,3,72.0,59.0,17.0,10.0
4,6411,37.732350,-122.414100,0.670,0,0,0,0,0,0,...,0,2018,17,10,9,22,57.0,77.0,5.8,10.0


Now we're ready to generate the submission file. The submission file needs the columns `ID` and `Severity` only.

In [38]:
test_df[['ID', 'Severity']].to_csv('/kaggle/working/submission.csv', index=False)

The remaining steps is to submit the generated file and are as follows. 

1. Press `Save Version` on the upper right corner of this notebook.
2. Write a `Version Name` of your choice and choose `Save & Run All (Commit)` then click `Save`.
3. Wait for the saved notebook to finish running the go to the saved notebook.
4. Scroll down until you see the output files then select the `submission.csv` file and click `Submit`.

Now your submission will be evaluated and your score will be updated on the leaderboard! CONGRATULATIONS!!

## Conclusion

In this notebook, we have demonstrated the essential steps that one should do in order to get "slightly" familiar with the data and the submission process. We chose not to go into details in each step to keep the welcoming notebook simple and make a room for improvement.

You're encourged to `Fork` the notebook, edit it, add your insights and use it to create your submission.